<font size=3 color='blue'>

# <center> Class 13-1, December 15, 2021


<font size=5 color="blue">

Connection Geometries in Neural Networks
    
<font size=4 color="black">
$$ $$ 
In the architecture of a neural network, it is important to know where the connections originate and terminate.

<font size=5 color="blue">

Neural Networks with shortcut connections


<font size=4 color="black">

In many problems, the contribution to the response of the input variables is often linear, and nonlinear effects often coincide with small deviations from a linear solution. Such a mixed contribution to the response can be handled with a neural network with shorcut connections; i. e. the input variable to a set of layers is also connected to the output of a set. The set of layers models the nonlinear effect associated to the input variable.

<img src="./images/FF-with-shortcuts.jpg" width=400 height=400 align = "left" >    
<img src="./images/FF-with-shortcuts-1.png" width=350 height=300 align = "center" >

<font size=4 color="black">
    
[Reference: Spirals separation](./literature/shortcuts_Learning-two-spirals-apart_1989.pdf)

<font size=2 color="black">

## ResNet

<img src="./images/resnet-keras.png" width=400 height=400 align = "center" >

<font size=4 color="black">
$$ $$
Residual Learning: a learning block. F(X) represents the nonlinear response of the input X.
$$ $$    

[Reference: ResNet](./literature/RCNN_2016.pdf)
    
[Comment: ResNet in Keras](https://towardsdatascience.com/understanding-and-coding-a-resnet-in-keras-446d7ff84d33)


<img src="./images/resnet-1.png" width=920 height=620 align = "center" >

<font size=4 color="black">
    
[Comment: ResNet](https://towardsdatascience.com/an-overview-of-resnet-and-its-variants-5281e2f56035)

 <font size=4 color="black">

With the architecture of the present notebook, we can construct ResNet networks with different number of Layers (indicated with the number).
$$ $$
ResNet-20 

ResNet-32
    
ResNet-44
    
ResNet-56
    
ResNet-110
    
ResNet-164
    
ResNet-1001
    
The execution time of the neural network depends on the number of layers.

<font size=5 color='blue'>
Applications

<font size=4 color='black'> 
    
[Referencia: Dynamical System Evolution](./literature/resnet_applications/Dynamical_system_evolution_2019.pdf)

<font size=4 color='black'> 
    
[Referencia: Health Monitoring](./literature/resnet_applications/health_monitoring_2020.pdf)

<font size=4 color='black'> 
    
[Referencia: Land Use Detection](./literature/resnet_applications/Land-use-detection_2019.pdf)

<font size=4 color='black'> 
    
[Referencia: Time Series Classification](./literature/resnet_applications/Timeseries_classification.pdf)

<font size=5 color="blue">

Extensions of ResNet

<font size=4 color="black">

[Reference: Adjustable shorcut connections](./literature/Improving_ResNet_2018.pdf)
    
[Reference: ResNeXt](./literature/ResNeXt_2017.pdf)

[Reference: DenseNet](./literature/DEnse_CNN2017.pdf)

<font size=5 color="blue">

Deep Learning: ResNet 

In [ ]:
import tensorflow as tf

physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("physical_devices-------------", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
from platform import python_version
import pkg_resources
def version_library(programa):
    return pkg_resources.get_distribution(programa).version
programas=['numpy', 'tensorflow','pydot', 'matplotlib', 'pickle']

import itertools
from itertools import product
for v in product(iter(programas)):
    print(v[0])
    try:
        print(version_library(v[0]))
    except Exception as e: print(e)

In [ ]:
from __future__ import print_function

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, Activation
from tensorflow.keras.layers import AveragePooling2D, Input, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
#from keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model

import numpy as np
import os

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

import time

In [ ]:
print("Numpy version", np.__version__)
print("TensorFlow version", tf.__version__)
print("Keras version", keras.__version__)
#print("Pydot version", pydot.__version__)
print("Matplotlib version", matplotlib.__version__)
#print("Pickle version", pickle.format_version)
from platform import python_version
print("Python version", python_version())

<font size=5 color='blue'>
Tensorboard with Jupyter Notebooks

In [ ]:
# Deleting possible active application of tensorboard
!kill $(ps -e | grep 'tensorboard' | awk '{print $1}')
!kill $(ps -e | grep 'tensorboard' | awk '{print $1}')

# Load the TensorBoard notebook extension
%load_ext tensorboard

# Clear any logs from previous runs
!rm -rf ./logs_resnet/


<img src="./images/Picture1.png" width=420 height=420 align = "center" >

<font size=5 color='blue'>
Object Recognition through images using CNN

<img src="./images/cifar-10.png" width=420 height=420 align = "center" >

<font size=5 color="blue">

CIFAR-10 database

<font size=4 color='black'>

Generation or extraction of the raw data
  

In [ ]:
# Number of classes
num_classes = 10

In [ ]:
from tensorflow.keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
# Input image dimensions.
input_shape = x_train.shape[1:]

print(input_shape)

<font size=5 color='blue'>

Analysis of the raw data

<font size=4 color='black'>   
 The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images. $$ $$

The dataset is divided into five training batches and one test batch, each training batch with 10000 images. The test batch contains exactly 1000 randomly-selected images from each class. The training batches contain the remaining images in random order, but some training batches may contain more images from one class than another. Between them, the training batches contain exactly 5000 images from each class.

The ten classes are:

| index | class name |
| --- | --- |
| 0 | airplane |
| 1 | automobile | 										
| 2 | bird |										
| 3 | cat |										
| 4 | deer |										
| 5 | dog |										
| 6 | frog |										
| 7 | horse |										
| 8 | ship |										
| 9 | truck|

<font size=4 color='black'>
    
We define a dictionary to associate the class number to a class name.



In [ ]:
# We define a dictionary to associate the class number to a class name.

dic = {0: 'airplane', 1: 'automobile', 2: 'bird', 3: 'cat', 4: 'deer', 
       5: 'dog', 6: 'frog', 7:'horse', 8: 'ship', 9: 'truck'}

<font size=4 color="black">
    
Viewing one sample from the data sets

In [ ]:
x_train[0][0]

<font size=4 color="black">
    
Every value of X in the example is between 0 and 255, they are not normalized.

In [ ]:
y_train[0]

<font size=4 color="black">
    
Next, we show a sample: its target and image.

In [ ]:
sample = 9

plt.imshow(x_train[sample]);

print('y =',  np.squeeze(y_train[sample]), ';', 'the sample', sample,\
      'corresponds to a', dic[int(np.squeeze(y_train[sample]))])

<font size=5 color='blue'>

Transformation of the raw data 

<font size=4 color="black">
$$ $$
To normalize the X values of an image, we divide each pixel value by 255

In [ ]:
# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [ ]:
# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

In [ ]:
# If subtract pixel mean is enabled

if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

In [ ]:
x_train[0][0]

In [ ]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

In [ ]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
y_train[0]

In [ ]:
print('y_train shape =', y_train.shape)

<font size=5 color='blue'>

Definition of the neural network architecture
  

<font size=4 color='black'>
    
Learning Rate Schedule

Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs. Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate

In [ ]:
def lr_schedule(epoch):

    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    
    return lr


<font size=4 color='black'>
    
2D Convolution-Batch Normalization-Activation stack builder

In [ ]:
def resnet_layer(inputs, num_filters=16, kernel_size=3, strides=1, activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    '''
        # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    '''
    
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


<font size=4 color='black'>


    
ResNet Version 1 Model, accoding to the following paper: [Model 1](./literature/RCNN_2016.pdf) 

    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number of filters and the
    same filter map sizes.
    
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    
    The Number of parameters is approx the same as Table 6 of paper cited above.
    
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M

In [ ]:
def resnet_v1(input_shape, depth, num_classes=10):
    
    '''
    # Arguments
    input_shape (tensor): shape of input image tensor
    depth (int): number of core convolutional layers
    num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
    model (Model): Keras model instance
    '''
    
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)
    print('The number of stages = ', num_res_blocks,'\n' )

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model


<font size=4 color='black'>
   
ResNet Version 2 Model. According to the following paper: [Model 2](./literature/Model2_2016.pdf)

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number of filters and the
    same filter map sizes.
    
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256

In [ ]:
def resnet_v2(input_shape, depth, num_classes=10):
    
    '''
    # Arguments
    input_shape (tensor): shape of input image tensor
    depth (int): number of core convolutional layers
    num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
    model (Model): Keras model instance
    '''
    
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)
    print('The number of stages = ', num_res_blocks, '\n')

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

<font size=5 color='blue'>

Generating a model of deep neural network 

<font size=4 color='black'>

Choosing between Model1 and Model2

In [ ]:
# Summary of the results described in the papers of Model1 and Model2
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------

# Model version

# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 1

# This is valid only for ResNet20. The respective values should be changed for another version of ResNet 

if version == 1:
    n = 3    # For Model ResNet20 version 1
elif version == 2:
    n = 2    # For Model ResNet20 version 2
    
    

In [ ]:
num_classes = 10

# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2


if version == 2:
    resnet_model = resnet_v2(input_shape=input_shape, depth=depth, num_classes = num_classes)
else:
    resnet_model = resnet_v1(input_shape=input_shape, depth=depth, num_classes = num_classes)

In [ ]:

print("The number of layers in depth is ", depth)
print("It excludes the pooling and dense (full connected) layers ")

In [ ]:
# Model name, depth and version

model_type = 'ResNet%dv%d' % (depth, version)

print(model_type)

In [ ]:
# Prepare model chacking saving directory.

save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)


In [ ]:
# Prepare model name
model_name_0 = 'cifar10_%s_model' % model_type
print(model_name_0)

In [ ]:

plot_model(resnet_model, to_file=model_name_0+'.png', show_shapes=True, rankdir='TB', expand_nested=True, show_layer_names=True, dpi=96)

In [ ]:
resnet_model.summary()

<font size=5 color='blue'>

Compiling the model 

In [ ]:
#Compiling the model using Adam as optimizer

resnet_model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=lr_schedule(0)),
              metrics=['accuracy'])


<font size=5 color='blue'>
Callbacks $$ $$
    
<font size=4 color='black'> 
    
[Keras.callbacks](https://www.tensorflow.org/guide/keras/custom_callback)

<font size=4>
A callback is a powerful tool to customize the behavior of a Keras model during training, evaluation, or inference. Examples include tf.keras.callbacks.TensorBoard to visualize training progress and results with TensorBoard, or tf.keras.callbacks.ModelCheckpoint to periodically save your model during training.

<font size=4>
All callbacks subclass the keras.callbacks.Callback class, and override a set of methods called at various stages of training, testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training.

You can pass a list of callbacks (as the keyword argument callbacks) to the following model methods:

    keras.Model.fit()
    keras.Model.evaluate()
    keras.Model.predict()


<font size=5 color='blue'>
Learning rate scheduling using callbacks

<font size=4>
In the next cell, we show how a custom Callback can be used to dynamically change the learning rate of the optimizer during the course of training.

In [ ]:
# Prepare callbacks for model saving and for learning rate adjustment.

checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

tensorboard_callback = keras.callbacks.TensorBoard(
    log_dir="logs_resnet/model_name_0", histogram_freq=1)


callbacks = [checkpoint, lr_reducer, lr_scheduler, tensorboard_callback]


<font size=5 color='blue'>

Running the model    

In [ ]:
# Training parameters

data_augmentation = False
batch_size = 32  # original paper trained all networks with batch_size=128
epochs = 3

In [ ]:
# Run training, with or without data augmentation.

start_time = time.time()



if not data_augmentation:
    print('Not using data augmentation.')
    history=resnet_model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)
    
    # Fit the model on the batches generated by datagen.flow().
    history=resnet_model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        validation_data=(x_test, y_test),
                        epochs=epochs, verbose=1, workers=4,
                        callbacks=callbacks + tensorboard_callback)
    
end_time = time.time()
print("Time for training: {:10.4f}s".format(end_time - start_time))

In [ ]:
resnet_model.save(model_name_0+'.h5')

In [ ]:
# Score trained model.
scores = resnet_model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


<font size=4 color="black">
    
* Note: if you run `fit()` again, the `model` will continue training, starting with the parameters it has already learnt, instead of reinitializing them.


<font size=5 color='blue'>

Plotting the loss function 

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Lr = 0.001, loss_train: 0.1665, \n loss_val: 0.4389, BatchNorm=True \n Data_augmentation=True')
plt.ylabel('Cost')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
#plt.ylim(top=13)    # The instruction is used to limit the upper value of the loss function 
#plt.ylim(bottom=0)  # The instruction is used to limit the lower value of the loss function
plt.show()

<font size=5 color='blue'>

Plotting the accuracy 

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Lr = 0.001, Acc_train: 0.9860, \n Acc_val: 0.9174 BatchNorm=True \n Data_augmentation=True')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()

<font size=5, color=blue>
    Data analysis

<font size=5, color=blue>
    With NETRON
$$ $$
<font size=4, color='black'>
    
[How to extract kernel values](https://shivang-ahd.medium.com/how-to-extract-kernel-values-in-cnn-using-netron-and-generate-feature-maps-82cdb6020bb0)

[NETRON](https://netron.app/)

<font size=5, color=blue>
    With Tensorboard

In [ ]:
%tensorboard --logdir logs_resnet/model_name_0

<font size=4 color="black">
    
[Summary of the runs for 200 epochs](Clase-13.odt)